In [ ]:

import os
import shutil

input_folder = "aligned_faces_agedb"
output_folder = "aligned_AgeDB_age"


image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]


for image in image_files:
        parts = image.split("_")
          
        label, name, age, gender_ext = parts[0], parts[1], parts[2], parts[3]
        
        age = int(age)

        gender = gender_ext.split('.')[0]
        age_group = f"{(age // 10) * 10}"
        name_gender_folder = f"{name}_{gender}"
        age_group_folder = os.path.join(output_folder, age_group) 
        person_folder = os.path.join(age_group_folder, name_gender_folder) 

        os.makedirs(person_folder, exist_ok=True)

        source_path = os.path.join(input_folder, image)
        destination_path = os.path.join(person_folder, image)


        shutil.copy(source_path, destination_path)





In [4]:
#genuine folder 생성 코드

import os
import shutil
from itertools import combinations

# 폴더 경로 
input_folder = "Asian_celebrity_align"

# 쌍을 저장할 폴더
output_folder = "gen"
os.makedirs(output_folder, exist_ok=True)  # gen 폴더 생성

# 전체 인덱스 초기화
global_index = 0

# 10대 폴더 내의 각 "이름_성별" 폴더를 순회
for person_folder in os.listdir(input_folder):
    person_path = os.path.join(input_folder, person_folder)

    # 폴더가 아니라면 건너뛰기
    if not os.path.isdir(person_path):
        continue

    # 해당 폴더 안의 이미지 파일 가져오기
    image_files = sorted([f for f in os.listdir(person_path) if f.endswith('.jpg')])

    # 가능한 모든 쌍 생성 (2개씩)
    # pairs = list(combinations(image_files, 2))  # 모든 조합 생성
    # 연속한 사진끼리 짝지음 단 홀수 면 마지막과 처음 짝지음
    pairs=[]
    if len(image_files) % 2 == 0:
        for i in range(0, len(image_files), 2):
            pairs.append((image_files[i], image_files[i+1]))
    else:
        for i in range(0, len(image_files)-1, 2):
            pairs.append((image_files[i], image_files[i+1]))
        pairs.append((image_files[-1],image_files[0]))
        
    # 쌍을 `gen/` 폴더에 저장
    for img1, img2 in pairs:
        pair_folder = os.path.join(output_folder, str(global_index))  #인덱스별 폴더 생성
        os.makedirs(pair_folder, exist_ok=True)

        # 원본 이미지 경로
        source1 = os.path.join(person_path, img1)
        source2 = os.path.join(person_path, img2)

        # 저장할 경로
        dest1 = os.path.join(pair_folder, f"pair1_{img1}")
        dest2 = os.path.join(pair_folder, f"pair2_{img2}")

        # 이미지 복사
        shutil.copy(source1, dest1)
        shutil.copy(source2, dest2)

        # 인덱스 증가
        global_index += 1




In [ ]:
#imposter folder 생성 코드
import os
import shutil
import random
from concurrent.futures import ThreadPoolExecutor

# 폴더 경로
input_folder = "Asian_celebrity_align"  

# 생성된 쌍을 저장할 폴더
output_folder = "imp"
os.makedirs(output_folder, exist_ok=True)

# 성별별로 폴더 정리
male_folders = []
female_folders = []
person_images = {}

# 사람별 이미지 리스트 저장 
for person_folder in os.listdir(input_folder):
    folder_path = os.path.join(input_folder, person_folder)
    if not os.path.isdir(folder_path):
        continue
    
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    if not image_files:
        continue  # 이미지가 없는 경우 스킵

    # 파일명에서 성별 추출 
    gender = image_files[0].split('_')[-1].split('.')[0]  # 확장자 제거 후 마지막 요소 추출

    # 성별을 구분하여 저장
    person_images[person_folder] = image_files
    if gender == 'm':
        male_folders.append(person_folder)
    elif gender == 'f':
        female_folders.append(person_folder)

#  무작위 쌍 생성
pair_list = []
gender_groups = [male_folders, female_folders]  # 성별 그룹

while len(pair_list) < 1:
    print(f"현재 gender_groups: {gender_groups}")  # 디버깅용
    if not gender_groups:
        print("❌ 모든 성별 그룹이 비어 있습니다. 종료합니다.")
        break
    gender_group = random.choice(gender_groups)  # 랜덤하게 성별 선택

    # 성별 그룹이 비어있는 경우 다시 선택 (무한 루프 방지)
    if len(gender_group) < 2:
        gender_groups.remove(gender_group)  # 비어있는 그룹 제거
        if not gender_groups:  # 모든 그룹이 비어 있다면 루프 종료
            print(" 더 이상 생성할 수 있는 쌍이 없습니다.")
            break
        continue  

    person1, person2 = random.sample(gender_group, 2)

    # 각 폴더에서 랜덤하게 1장씩 이미지 선택
    if not person_images[person1] or not person_images[person2]:
        continue  # 이미지가 없는 경우 다시 시도

    img1 = random.choice(person_images[person1])
    img2 = random.choice(person_images[person2])

    pair_list.append((person1, img1, person2, img2))


# 멀티스레딩을 이용해 빠르게 파일 복사
def copy_images(pair_data, idx):
    person1, img1, person2, img2 = pair_data
    pair_folder = os.path.join(output_folder, str(idx))
    os.makedirs(pair_folder, exist_ok=True)

    # 원본 이미지 경로
    source1 = os.path.join(input_folder, person1, img1)
    source2 = os.path.join(input_folder, person2, img2)

    # 저장할 경로
    dest1 = os.path.join(pair_folder, f"pair1_{img1}")
    dest2 = os.path.join(pair_folder, f"pair2_{img2}")

    shutil.copy(source1, dest1)
    shutil.copy(source2, dest2)

# 병렬로 파일 복사 실행
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(copy_images, pair_list, range(len(pair_list)))



In [ ]:
import os
import shutil
import random
from concurrent.futures import ThreadPoolExecutor

# 폴더 경로
input_folder = "Asian_celebrity_align"
output_folder = "imp"
os.makedirs(output_folder, exist_ok=True)

# 성별별 폴더 정리
male_folders = []
female_folders = []
person_images = {}

# 사람별 이미지 리스트 저장
for person_folder in os.listdir(input_folder):
    folder_path = os.path.join(input_folder, person_folder)
    if not os.path.isdir(folder_path):
        continue
    
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    if not image_files:
        continue

    gender = image_files[0].split('_')[-1].split('.')[0]

    person_images[person_folder] = image_files
    if gender == 'm':
        male_folders.append(person_folder)
    elif gender == 'f':
        female_folders.append(person_folder)

# 무작위 쌍 생성
pair_list = []

# 남성, 여성 중 이미지가 있는 성별만 골라낸다.
gender_groups = []
if len(male_folders) >= 2:
    gender_groups.append(male_folders)
if len(female_folders) >= 2:
    gender_groups.append(female_folders)

if not gender_groups:
    print("❌ 충분한 데이터가 없습니다. (성별에 관계없이 최소 두 명 필요)")
else:
    while len(pair_list) < 1:
        gender_group = random.choice(gender_groups)
        person1, person2 = random.sample(gender_group, 2)

        if not person_images.get(person1) or not person_images.get(person2):
            continue

        img1 = random.choice(person_images[person1])
        img2 = random.choice(person_images[person2])

        pair_list.append((person1, img1, person2, img2))


# 파일 복사 함수
def copy_images(pair_data, idx):
    try:
        person1, img1, person2, img2 = pair_data
        pair_folder = os.path.join(output_folder, str(idx))
        os.makedirs(pair_folder, exist_ok=True)

        source1 = os.path.join(input_folder, person1, img1)
        source2 = os.path.join(input_folder, person2, img2)

        dest1 = os.path.join(pair_folder, f"pair1_{img1}")
        dest2 = os.path.join(pair_folder, f"pair2_{img2}")

        shutil.copy(source1, dest1)
        shutil.copy(source2, dest2)

        print(f"✅ {img1} & {img2} copied to {pair_folder}")

    except Exception as e:
        print(f"❌ 오류 발생 (파일 복사 중): {e}")

# 멀티스레딩 실행
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(copy_images, pair, idx) for idx, pair in enumerate(pair_list)]
    
    for future in futures:
        future.result()  # 실행 중 오류 출력
